In [68]:
import numpy as np
import pandas as pd

In [1]:
import requests
import urllib.request
import urllib.parse
import json
from bs4 import BeautifulSoup

In [3]:
from search import *
import song_database_generator as sdg

In [34]:
base = "https://api.genius.com"

path_cred = "/Users/pauserrabergeron/Private/Ironhack/Course/credentials/"
f = open(path_cred + "genius_credentials.txt", "r")
dict_credentials = json.loads(f.read()[:-1])

In [5]:
dict_credentials

{'client_id': '-q1tRGBZMEOk6JewZCC_KWZBxyFSg9nccGlX11Cb3MxpGpzWG4FBSJIXCJS33D3x'}

In [6]:
def get_json(path, params=None, headers=None):
    '''Send request and get response in json format.'''

    # Generate request URL
    requrl = '/'.join([base, path])
    token = "Bearer {}".format(dict_credentials["client_id"])
    if headers:
        headers['Authorization'] = token
    else:
        headers = {"Authorization": token}

    # Get response object from querying genius api
    response = requests.get(url=requrl, params=params, headers=headers)
    response.raise_for_status()
    return response.json()

In [26]:
def search(song_name):
    '''Search Genius API via song & artist name.'''
    search = "/search?q="
    query = base + search + urllib.parse.quote(artist_name)
    request = urllib.request.Request(query)

    request.add_header("Authorization", "Bearer " + dict_credentials["client_id"])
    request.add_header("User-Agent", "")

    response = urllib.request.urlopen(request, timeout=3)
    raw = response.read()
    data = json.loads(raw)['response']['hits']

    for item in data:
        # Print the artist and title of each result
        print(item['result']['primary_artist']['name']
              + ': ' + item['result']['title'])

In [8]:
def connect_lyrics(song_id):
    '''Constructs the path of song lyrics.'''
    url = "songs/{}".format(song_id)
    data = get_json(url)

    # Gets the path of song lyrics
    path = data['response']['song']['path']

    return path

In [9]:
def search_artist(artist_id):
    '''Search meta data about artist Genius API via Artist ID.'''
    search = "artists/"
    path = search + str(artist_id)
    request = get_json(path)
    data = request['response']['artist']

    print(data["followers_count"])
    # Lots of information we can scrape regarding the artist, check keys
    return data["followers_count"] # number of followers

In [10]:
def retrieve_lyrics(song_id):
    '''Retrieves lyrics from html page.'''
    path = connect_lyrics(song_id)

    URL = "http://genius.com" + path
    page = requests.get(URL)

    # Extract the page's HTML as a string
    html = BeautifulSoup(page.text, "html.parser")

    # Scrape the song lyrics from the HTML
    lyrics = html.find("div", class_="lyrics").get_text()
    return lyrics

In [32]:
def get_song_id(artist_id):
    '''Get all the song id from an artist.'''
    current_page = 1
    next_page = True
    songs = [] # to store final song ids

    while next_page:
        path = "artists/{}/songs/".format(artist_id)
        params = {'page': current_page} # the current page
        data = get_json(path=path, params=params) # get json of songs

        page_songs = data['response']['songs']
        if page_songs:
            # Add all the songs of current page
            songs += page_songs
            # Increment current_page value for next loop
            current_page += 1
            print("Page {} finished scraping".format(current_page))
            # If you don't wanna wait too long to scrape, un-comment this
            if current_page == 2:
                break

        else:
            # If page_songs is empty, quit
            next_page = False

    print("Song id were scraped from {} pages".format(current_page))

    # Get all the song ids, excluding not-primary-artist songs.
    songs = [song["id"] for song in songs
            if song["primary_artist"]["id"] == artist_id]

    return songs

In [12]:
def get_song_information(song_ids):
    '''Retrieve meta data about a song.'''
    # initialize a dictionary.
    song_list = {}
    print("Scraping song information")
    for i, song_id in enumerate(song_ids):
        print("id:" + str(song_id) + " start. ->")

        path = "songs/{}".format(song_id)
        data = get_json(path=path)["response"]["song"]

        song_list.update({
        i: {
            "title": data["title"],
            "album": data["album"]["name"] if data["album"] else "<single>",
            "release_date": data["release_date"] if data["release_date"] else "unidentified",
            "featured_artists":
                [feat["name"] if data["featured_artists"] else "" for feat in data["featured_artists"]],
            "producer_artists":
                [feat["name"] if data["producer_artists"] else "" for feat in data["producer_artists"]],
            "writer_artists":
                [feat["name"] if data["writer_artists"] else "" for feat in data["writer_artists"]],
            "genius_track_id": song_id,
            "genius_album_id": data["album"]["id"] if data["album"] else "none"}
        })

        print("-> id:" + str(song_id) + " is finished. \n")
    return song_list

In [13]:
def main():
    # Example searches
    term = 'Kanye West'
    artist_id = 72

    # Grabs all song id's from artist
    songs_ids = get_song_id(72)

    # Get meta information about songs
    #song_list = get_song_information(songs_ids)

    # Scrape lyrics from the songs
    song_lyrics = [retrieve_lyrics(song_id) for song_id in songs_ids]

    for lyrics in song_lyrics:
        print(lyrics)


    # Gets information regarding the artist themself
    # followers = search_artist(artist_id)

    # Shows some random songs from arist and lyrics
    #search(term)

In [35]:
main()

Page 2 finished scraping
Song id were scraped from 2 pages


AttributeError: 'NoneType' object has no attribute 'get_text'

In [29]:
search = "search?q="
query = base + search + urllib.parse.quote("7 Days Craig David")
request = urllib.request.Request(query)

request.add_header("Authorization", "Bearer " + dict_credentials["client_id"])
request.add_header("User-Agent", "")

response = urllib.request.urlopen(request, timeout=3)
raw = response.read()
data = json.loads(raw)['response']['hits']

In [30]:
data

[{'highlights': [],
  'index': 'song',
  'type': 'song',
  'result': {'annotation_count': 3,
   'api_path': '/songs/110040',
   'full_title': '7 Days by\xa0Craig\xa0David',
   'header_image_thumbnail_url': 'https://images.genius.com/96e2732a31a288d1e0c04451107ced68.300x300x1.jpg',
   'header_image_url': 'https://images.genius.com/96e2732a31a288d1e0c04451107ced68.519x519x1.jpg',
   'id': 110040,
   'lyrics_owner_id': 46871,
   'lyrics_state': 'complete',
   'path': '/Craig-david-7-days-lyrics',
   'pyongs_count': 25,
   'song_art_image_thumbnail_url': 'https://images.genius.com/96e2732a31a288d1e0c04451107ced68.300x300x1.jpg',
   'song_art_image_url': 'https://images.genius.com/96e2732a31a288d1e0c04451107ced68.519x519x1.jpg',
   'stats': {'unreviewed_annotations': 0, 'hot': False, 'pageviews': 46866},
   'title': '7 Days',
   'title_with_featured': '7 Days',
   'url': 'https://genius.com/Craig-david-7-days-lyrics',
   'primary_artist': {'api_path': '/artists/680',
    'header_image_url':

In [37]:
path = connect_lyrics(110040)

URL = "http://genius.com" + path
page = requests.get(URL)

# Extract the page's HTML as a string
html = BeautifulSoup(page.text, "html.parser")

# Scrape the song lyrics from the HTML
lyrics = html.find("div", class_="lyrics").get_text()
lyrics

'\n\n[Intro]\nYou know what?\nGot something to say\nCraig David\n7 days\nCheck it out, yeah\n\n[Verse 1]\nOn my way to see my friends\nWho lived a couple blocks away from me (Oh)\nAs I walked through the subway\nIt must have been about quarter past three\nIn front of me\nStood a beautiful honey with a beautiful body (Body)\nShe asked me for the time\nI said it\'d cost her a name\nA six digit number and a date with me tomorrow at nine\n\n[Pre-Chorus]\nDid she decline? No\nDidn\'t she mind? I don\'t think so\nWas it for real? Damn sure\nWhat was the deal? A pretty girl, age 24\nSo was she keen? She couldn\'t wait\nCinnamon queen? Let me update\nWhat did she say?\nShe said she\'d, "Love to rendezvous"\nShe asked me, "What we were gonna do?"\nSaid we\'d start with a bottle of Moet for two\n\n[Chorus]\nMonday\nTook her for a drink on Tuesday\nWe were making love by Wednesday\nAnd on Thursday and Friday and Saturday, we chilled on Sunday\nI met this girl on Monday\nTook her for a drink on Tu

In [39]:
URL

'http://genius.com/Craig-david-7-days-lyrics'

In [63]:
def search(song_name):
    '''Search Genius API via song & artist name.'''
    search = "/search?q="
    query = base + search + urllib.parse.quote(song_name)
    request = urllib.request.Request(query)

    request.add_header("Authorization", "Bearer " + dict_credentials["client_id"])
    request.add_header("User-Agent", "")

    response = urllib.request.urlopen(request, timeout=3)
    raw = response.read()
    data = json.loads(raw)['response']['hits']

    for item in data:
        # Print the artist and title of each result
        print(item['result']['primary_artist']['name']
              + ': ' + item['result']['title'] + " --> " + str(item['result']['id']))

In [64]:
search("Vogue Madonna")

Madonna: Vogue --> 189590
Saturday Night Live: Christmastime for the Jews --> 4136345
Pimp C: Slab Music --> 2385796
Madonna: Holy Water / Vogue (Live) --> 3201129
Madonna: Vogue - MDNA World Tour / Live 2012 --> 4961638
Madonna: Vogue 2008 - Live --> 5016041
Madonna: Vogue (Live 2004) --> 5502643
Genius Brasil Traduções: Madonna - Vogue (Tradução em Português) --> 6148121
Genius Brasil Traduções: Madonna - Vogue - MDNA World Tour / Live 2012 (Tradução em Português) --> 6249063


In [51]:
data[0]['result']['id']

110040

In [66]:
retrieve_lyrics(5016041)

"\n\n[Intro: Madonna, Justin Timberlake]\nTick tock, tick tock, tick tock\nMadonna\nTick tock, tick tock, tick tock\nMadonna\nTick tock, tick tock, tick\u2005tock\nTick\u2005tock, tick tock,\u2005tick tock, tick tock\nMadonna\nLadies\nLadies\nLadies, ladies, ladies,\u2005ladies, ladies\nLay-lay-ladies with an attitude\nFellas that were in the mood\nDon't just stand there\nDon't don't don't just stand there, let's get to it\nDon't just stand there\n(Tick tock, tick tock, tick tock, tick tock)\nStrike a pose there's nothing to it\nVogue\n\n[Verse 1: Madonna]\nLook around, everywhere you turn is heartache\nIt's everywhere that you go (Look around)\nYou try everything you can to escape\nThe pain of life that you know (Life that you know)\nIf all else fails and you long to be\nSomething better than you are today\nI know a place where you can get away\nIt's called a dance floor, and here's what it's for, so\n\n[Chorus: Madonna]\nCome on, vogue\nLet your body move to the music\nHey, hey, hey\

In [55]:
'''Retrieves lyrics from html page.'''
path = connect_lyrics(110040)

URL = "http://genius.com" + path
page = requests.get(URL)

    # Extract the page's HTML as a string
html = BeautifulSoup(page.text, "html.parser")

    # Scrape the song lyrics from the HTML
lyrics = html.find("div", class_="lyrics").get_text()
lyrics

'\n\n[Intro]\nYou know what?\nGot something to say\nCraig David\n7 days\nCheck it out, yeah\n\n[Verse 1]\nOn my way to see my friends\nWho lived a couple blocks away from me (Oh)\nAs I walked through the subway\nIt must have been about quarter past three\nIn front of me\nStood a beautiful honey with a beautiful body (Body)\nShe asked me for the time\nI said it\'d cost her a name\nA six digit number and a date with me tomorrow at nine\n\n[Pre-Chorus]\nDid she decline? No\nDidn\'t she mind? I don\'t think so\nWas it for real? Damn sure\nWhat was the deal? A pretty girl, age 24\nSo was she keen? She couldn\'t wait\nCinnamon queen? Let me update\nWhat did she say?\nShe said she\'d, "Love to rendezvous"\nShe asked me, "What we were gonna do?"\nSaid we\'d start with a bottle of Moet for two\n\n[Chorus]\nMonday\nTook her for a drink on Tuesday\nWe were making love by Wednesday\nAnd on Thursday and Friday and Saturday, we chilled on Sunday\nI met this girl on Monday\nTook her for a drink on Tu

In [54]:
html

<!DOCTYPE html>

<html>
<head>
<title>Craig David – 7 Days Lyrics | Genius Lyrics</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="app-id=709482991" name="apple-itunes-app"/>
<link href="https://assets.genius.com/images/apple-touch-icon.png?1613147657" rel="apple-touch-icon"/>
<link href="https://assets.genius.com/images/apple-touch-icon.png?1613147657" rel="apple-touch-icon"/>
<!-- Mobile IE allows us to activate ClearType technology for smoothing fonts for easy reading -->
<meta content="on" http-equiv="cleartype"/>
<meta content="f63347d284f184b0" name="y_key"/>
<meta content="Genius" property="og:site_name">
<meta content="265539304824" property="fb:app_id">
<meta content="308252472676410" property="fb:pages">
<link href="https://genius.com/opensearch.xml" rel="search" title="Genius" type="application/opensearchdescription+xml"/>
<script>
!function(){if('PerformanceLongTa

In [69]:
songs = pd.read_csv("../data/songs_all.csv")

In [70]:
songs

,name,artist_1,artist_2,uri,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Walking Away,Craig David,NaN,spotify:track:3R7fjB38qajI6JR69y5k4e,204867,0.596,0.686,9,-6.371,0,0.0345,0.19600,0.000000,0.1210,0.815,172.835
1,Hotel California - 2013 Remaster,Eagles,NaN,spotify:track:40riOy7x9W7GXjyGp4pjAv,391376,0.579,0.508,2,-9.484,1,0.0270,0.00574,0.000494,0.0575,0.609,147.125
2,I'm On Fire,Bruce Springsteen,NaN,spotify:track:3PzsbWSQdLCKDLxn7YZfkM,155880,0.620,0.446,1,-14.451,0,0.0347,0.58900,0.029900,0.0727,0.868,88.591


In [75]:
search("Hotel California "+songs.artist_1[1])

Eagles: Hotel California --> 1060
MEYTAL: Hotel California (Eagles Cover) --> 4394246
Michael Marc: Hotel California (Eagles - Instrumental) --> 2203856
Genius Lists: Songs That Reference Drugs --> 1453509
Brock Chenier: Vinyl Collection --> 1784680
Genius Traductions Françaises: Eagles - Hotel California (Traduction Française) --> 257633
Eagles: Hotel California (Live Millennium Concert Version) --> 2115103
Eagles: Hotel California (Orbitally Ambient Mix) --> 5268632
Genius Türkçe Çeviri: Eagles - Hotel California (Türkçe Çeviri) --> 5737022
Rolling Stone: The 500 Greatest Songs of All Time --> 494663


In [78]:
import lyricsgenius

ModuleNotFoundError: No module named 'lyricsgenius'